In [1]:
import requests
import json
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
import soundfile as sf
import random
import shutil
from pathlib import Path

In [2]:
with open('D:/studia/bird_sound_recognition/spec_name.json', "r") as file:
    spec_names = json.load(file)  

In [10]:
for specie in spec_names:
    temp = specie.split()

spec_names.sort()
spec_names

['Alauda arvensis',
 'Apus apus',
 'Bombycilla garrulus',
 'Ciconia ciconia',
 'Corvus cornix',
 'Cyanistes caeruleus',
 'Fringilla coelebs',
 'Hirundo rustica',
 'Luscinia megarhynchos',
 'Passer domesticus',
 'Phoenicurus ochruros',
 'Phylloscopus collybita',
 'Picus viridis',
 'Streptopelia decaocto',
 'Turdus merula']

In [21]:
target_sample_rate = 22500  # Docelowy sampling rate
# for specie in ["Trash file"]:  # Upewnij się, że spec_names jest zdefiniowane
#     temp = specie.split()
input_folder = f'D:\studia/bird_sound_recognition\data/recordings/unmodified/audio'
output_folder = f'D:\studia/bird_sound_recognitiondata/recordings/preprocessed/1_resampled_22-5kHz/Trash-class'
os.makedirs(output_folder, exist_ok=True)  # Tworzy folder wyjściowy, jeśli nie istnieje

# Przetwarzanie każdego pliku
for file_name in os.listdir(input_folder):
    if file_name.endswith(".wav"):
        input_path = os.path.join(input_folder, file_name)
        output_path = os.path.join(output_folder, os.path.splitext(file_name)[0] + ".wav")

        try:
            # Wczytanie pliku MP3
            audio, sr = librosa.load(input_path, sr=target_sample_rate)  # Resample do target_sample_rate

            # Zapisanie pliku WAV z nowym sampling rate
            sf.write(output_path, audio, target_sample_rate)

        except Exception as e:
            # Wyświetlenie błędu i nazwy problematycznego pliku
            print(f"Błąd podczas przetwarzania pliku: {input_path}")
            print(f"Typ błędu: {type(e).__name__}, szczegóły: {e}")


In [9]:
os.listdir(input_folder)[1]

'Apus-apus'

In [17]:
#foldery na nagrania
for specie in spec_names:
    temp = specie.split()
    folder = os.path.join('data/recordings/preprocessed/1_resampled_22-5kHz', f'{temp[0]}-{temp[1]}')
    os.makedirs(folder, exist_ok=True)

In [12]:
input_base_folder = "data/recordings/preprocessed/1_resampled_22-5kHz"
output_base_folder = "data/recordings/preprocessed/2_10-sec_len"
target_duration = 10  # Docelowa długość w sekundach

# Tworzenie folderu wyjściowego, jeśli nie istnieje
os.makedirs(output_base_folder, exist_ok=True)

# Pobranie listy gatunków (nazwy podfolderów w input_base_folder)
species_folders = [f for f in os.listdir(input_base_folder) if os.path.isdir(os.path.join(input_base_folder, f))]

# Przetwarzanie nagrań dla każdego gatunku
for species in ["Trash-file"]:
    input_folder = os.path.join(input_base_folder, species)
    output_folder = os.path.join(output_base_folder, species)
    os.makedirs(output_folder, exist_ok=True)  # Tworzy folder dla danego gatunku, jeśli nie istnieje

    # Przetwarzanie plików w folderze danego gatunku
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".wav"):  # Tylko pliki WAV
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, file_name)

            try:
                # Wczytanie pliku audio
                audio, sr = librosa.load(input_path, sr=None)

                # Obliczenie liczby próbek odpowiadającej 10 sekundom
                max_samples = int(target_duration * sr)

                # Obcinanie lub utrzymywanie oryginalnego rozmiaru
                trimmed_audio = audio[:max_samples]

                # Zapisanie obciętego pliku
                sf.write(output_path, trimmed_audio, sr)

            except Exception as e:
                # Wyświetlenie błędu i nazwy problematycznego pliku
                print(f"Błąd podczas przetwarzania pliku: {input_path}")
                print(f"Typ błędu: {type(e).__name__}, szczegóły: {e}")

In [22]:
# Parametry filtru pasmowego
low_cutoff = 400  # Dolna granica pasma (Hz)
high_cutoff = 10000  # Górna granica pasma (Hz)
sample_rate = 22500  # Częstotliwość próbkowania (Hz)
order = 5  # Rząd filtru

# Funkcja do zastosowania filtru pasmowego
def bandpass_filter(audio, sr, low_cutoff, high_cutoff, order=5):
    nyquist = 0.5 * sr  # Częstotliwość Nyquista
    low = low_cutoff / nyquist
    high = high_cutoff / nyquist
    b, a = signal.butter(order, [low, high], btype='band')
    filtered_audio = signal.filtfilt(b, a, audio)
    return filtered_audio

# Foldery wejściowe i wyjściowe
input_folder = f'D:\studia/bird_sound_recognition\data/recordings/unmodified/audio'
output_folder = f'D:\studia/bird_sound_recognition\data/recordings/preprocessed/3_bandpass-filter/Trash-class'

# Tworzenie folderu wyjściowego, jeśli nie istnieje
os.makedirs(output_base_folder, exist_ok=True)

# Iteracja po gatunkach (podfolderach w folderze wejściowym)
# for species in os.listdir(input_base_folder):
#     input_folder = os.path.join(input_base_folder, species)
#     output_folder = os.path.join(output_base_folder, species)

# Tworzenie folderu wyjściowego dla gatunku
os.makedirs(output_folder, exist_ok=True)

# Iteracja po plikach w folderze gatunku
for file_name in os.listdir(input_folder):
    if file_name.endswith(".wav"):
        input_path = os.path.join(input_folder, file_name)
        output_path = os.path.join(output_folder, file_name)

        try:
            # Wczytanie pliku audio
            audio, sr = librosa.load(input_path, sr=sample_rate)

            # Zastosowanie filtru pasmowego
            filtered_audio = bandpass_filter(audio, sr, low_cutoff, high_cutoff, order)

            # Zapisanie przefiltrowanego pliku
            sf.write(output_path, filtered_audio, sr)

            # print(f"Przetworzono {file_name} -> {output_path}")
        except Exception as e:
            print(f"Błąd przy przetwarzaniu {input_path}: {e}")

In [12]:
list = os.listdir("D:\studia/bird_sound_recognition\data/recordings\preprocessed/3_bandpass-filter")
list

['Alauda-arvensis',
 'Apus-apus',
 'Bombycilla-garrulus',
 'Ciconia-ciconia',
 'Corvus-cornix',
 'Cyanistes-caeruleus',
 'Fringilla-coelebs',
 'Hirundo-rustica',
 'Luscinia-megarhynchos',
 'Passer-domesticus',
 'Phoenicurus-ochruros',
 'Phylloscopus-collybita',
 'Picus-viridis',
 'Streptopelia-decaocto',
 'Trash-class',
 'Turdus-merula']

In [24]:
# Parametry Melspektrogramu
n_mels = 512  # Liczba pasm Mel
n_fft = 2048  # Rozmiar okna FFT
hop_length = 512  # Przesunięcie okna
win_length = 2048  # Długość okna
fmin = 20  # Minimalna częstotliwość
fmax = 11025  # Maksymalna częstotliwość (1/2 częstotliwości próbkowania)

# Parametry obrazu
image_width = 438  # Liczba klatek
image_height = 256  # Liczba pasm Mel

# Foldery wejściowe i wyjściowe
input_base_folder = "D:\studia/bird_sound_recognition\data/recordings\preprocessed/3_bandpass-filter"
output_base_folder = f"D:\studia/bird_sound_recognition\data\melspectrograms/{n_mels}_mel-bands"

# Tworzenie folderu wyjściowego, jeśli nie istnieje
os.makedirs(output_base_folder, exist_ok=True)
list = os.listdir(input_base_folder)

# Iteracja po gatunkach (podfolderach w folderze wejściowym)
for specie in list[13]:
    print(specie)
    specie = 'Streptopelia-decaocto'
    input_folder = os.path.join(input_base_folder, specie)
    output_folder = os.path.join(output_base_folder, specie)

    # Tworzenie folderu wyjściowego dla gatunku
    os.makedirs(output_folder, exist_ok=True)
    
    # Iteracja po plikach w folderze gatunku
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".wav"):
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, os.path.splitext(file_name)[0] + ".png")
    
            try:
                # Wczytanie pliku audio
                audio, sr = librosa.load(input_path, sr=22500)
    
                # Obliczanie melspektrogramu
                mel_spectrogram = librosa.feature.melspectrogram(
                    y=audio, sr=sr, n_mels=n_mels, n_fft=n_fft,
                    hop_length=hop_length, win_length=win_length, fmin=fmin, fmax=fmax
                )
    
                # Przekształcenie melspektrogramu na dB
                mel_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
    
                # Rysowanie melspektrogramu bez legendy (bez colorbar)
                plt.figure(figsize=(image_width / 100, image_height / 100), dpi=100)
                librosa.display.specshow(mel_db, x_axis='time', y_axis='mel', sr=sr)
    
                # Usunięcie paska kolorów (colorbar)
                plt.axis('off')  # Usunięcie osi
                plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Usunięcie marginesów
    
                # Zapisanie obrazu bez paska kolorów
                plt.savefig(output_path, bbox_inches='tight', pad_inches=0, transparent=True)
                plt.close()
    
                # print(f"Przetworzono {file_name} -> {output_path}")
            except Exception as e:
                print(f"Błąd przy przetwarzaniu {input_path}: {e}")


S
t
r


KeyboardInterrupt: 

In [2]:
n_mels = 512  # Liczba mel-bands (można dostosować)
# Proporcje podziału na zbiory
train_ratio = 0.75
val_ratio = 0.15
test_ratio = 0.10

In [3]:
# Ustawienia
input_folder = Path(f"D:/studia/bird_sound_recognition/data/melspectrograms/{n_mels}_mel-bands")
output_folder = Path(f"D:/studia/bird_sound_recognition/data/training2/split_{n_mels}-mel")

# Upewnij się, że folder wyjściowy istnieje
for split in ['train', 'val', 'test']:
    (output_folder / split).mkdir(parents=True, exist_ok=True)

# Iteracja po klasach (folderach)
for class_folder in input_folder.iterdir():
    if class_folder.is_dir():
        class_name = class_folder.name
        files = list(class_folder.glob("*.png"))  # Zakładamy, że pliki są w formacie .png

        # Wymieszaj pliki losowo
        random.shuffle(files)

        # Oblicz liczby plików dla każdego zbioru
        num_files = len(files)
        train_count = int(train_ratio * num_files)
        val_count = int(val_ratio * num_files)
        test_count = num_files - train_count - val_count  # Pozostałe pliki idą na test

        # Podziel pliki na treningowy, walidacyjny i testowy
        train_files = files[:train_count]
        val_files = files[train_count:train_count + val_count]
        test_files = files[train_count + val_count:]

        # Kopiowanie plików do odpowiednich folderów
        for split, split_files in zip(['train', 'val', 'test'], [train_files, val_files, test_files]):
            split_class_folder = output_folder / split / class_name
            split_class_folder.mkdir(parents=True, exist_ok=True)

            for file in split_files:
                shutil.copy(file, split_class_folder)

print("Podział danych zakończony!")


Podział danych zakończony!
